In [ ]:
for i in range(5):
    print(i)

In [ ]:
from sentence_transformers import SentenceTransformer

# 预先下载并保存嵌入模型
embedding_model_name = "all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(embedding_model_name)
embedding_model.save("model_embedding")


# RAG with Langchain

In [7]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
import fitz  # PyMuPDF
from langchain.document_loaders import SimpleDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

# 指定PDF文件目录
pdf_dir = "data_pdf"

# 提取PDF文本
def extract_text_from_pdfs(pdf_dir):
    texts = []
    for pdf_file in os.listdir(pdf_dir):
        if pdf_file.endswith(".pdf"):
            pdf_path = os.path.join(pdf_dir, pdf_file)
            doc = fitz.open(pdf_path)
            text = ""
            for page in doc:
                text += page.get_text()
            texts.append(text)
    return texts

# 加载PDF文本
texts = extract_text_from_pdfs(pdf_dir)

# 切分文本
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = text_splitter.create_documents(texts)

embedding_model =HuggingFaceEmbeddings("model_embedding/all-MiniLM-L6-v2")

# model =

# 生成嵌入向量并存储在FAISS向量数据库中
vector_store = FAISS.from_documents(docs, embedding_model)
vector_store.save_local("faiss_index")

ImportError: cannot import name 'SimpleDirectoryLoader' from 'langchain.document_loaders' (/usr/local/lib/python3.10/site-packages/langchain/document_loaders/__init__.py)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 加载Qwen2-7B模型和分词器
model_name = "qwen/autodl-tmp/qwen/Qwen2-7B-Instruct/"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

from langchain.chains import RetrievalAugmentedGenerationChain
from langchain.prompts import PromptTemplate
from langchain.llms import TransformersLLM
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

# 加载FAISS向量数据库
vector_store = FAISS.load_local("faiss_index", embedding_model)

# 定义Prompt模板
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="基于以下内容回答问题：\n\n{context}\n\n问题：{question}\n答案："
)

# 定义检索器
retriever = vector_store.as_retriever()

# 创建LLM对象
llm = TransformersLLM(model=model, tokenizer=tokenizer)

# 创建RAG链
rag_chain = RetrievalAugmentedGenerationChain(
    llm=llm,
    retriever=retriever,
    prompt_template=prompt_template,
    memory=ConversationBufferMemory()
)



 - [ ] _list1_